In [20]:
import numpy as np
from IPython.display import HTML 
HTML('../style/course.css')
HTML('../style/code_toggle.html')
from mirpy import miriad

# Continuum subtraction

The analysis and interpretation of spectral-line data is greatly simplified if all sources of continuum emission have been removed from the data. This chapter describes the methods that can be used to subtract such radio continuum emission from visibility datasets or datacubes, resulting in data that contain spectral-line emission and/or absorption only. Here we will adopt the following notation:
- $S(l,m,\nu)$ is the sky brightness as a function of position (relative to a reference position $l_0,m_0$, which we assume to be both the pointing and phase-tracking centre) and frequency;
- $A(l,m,\nu)$ is the primary beam pattern;
- $I(l,m,\nu) = S(l,m,\nu) \cdot A(l,m,\nu)$ is the apparent sky brightness;
- $B(l,m,\nu)$ is the point spread function or PSF;
- $I^\mathrm{D}(l,m,\nu)$ is the dirty cube obtained by convolving $I(l,m,\nu)$ with $B(l,m,\nu)$;
- $V(u,v,\nu)$ is the complex visibility;
- cubes and visibilities are composed of a continuum and a spectral line term, e.g., $I(l,m,\nu) = I_\mathrm{c}(l,m,\nu) + I_\mathrm{s}(l,m,\nu)$ and $V(u,v,\nu) = V_\mathrm{c}(u,v,\nu) + V_\mathrm{s}(u,v,\nu)$.

## Outline

**1.** <a href="http://localhost:8888/notebooks/contsub.ipynb#1.-Overview-of-continuum-subtraction-methods">Overview of continuum subtraction methods</a>

**2.** <a href="http://localhost:8888/notebooks/contsub.ipynb#2.-Cube-based-continuum-subtraction">Cube-based continuum subtraction</a>

   **2.1.** <a href="http://localhost:8888/notebooks/contsub.ipynb#2.1.-Working-with-dirty-cubes">Working with dirty cubes</a>

   **2.2.** <a href="http://localhost:8888/notebooks/contsub.ipynb#2.2.-Line-free-channel-selection-and-deconvolved-cubes">Line-free channel selection and deconvolved cubes</a>


**3.** <a href="http://localhost:8888/notebooks/contsub.ipynb#3.-Visibility-based-continuum-subtraction">Visibility-based continuum subtraction</a>

**4.** <a href="http://localhost:8888/notebooks/contsub.ipynb#4.-Model-based-continuum-subtraction">Model-based continuum subtraction</a>



## 1. Overview of continuum subtraction methods

Continuum emission can be removed from interferometric data in a variety of ways, which can be grouped under the following 3 categories:
- *cube based*, where the continuum emission is estimated and subtracted from a dirty cube $I^\mathrm{D}(l,m,\nu)$ or a clean cube $I(l,m,\nu)$;
- *visibility based*, where the continuum emission is estimated and subtracted from visibility spectra $V(u,v,\nu)$
- *model based*, wheren the continuum emission $I_\mathrm{c}(l,m,\nu)$ is modelled from a continuum (possibly multi-frequency) image, and the model is Fourier transformed and subtracted from the visibilities $V(u,v,\nu)$.

Each method has advantages and disadgantages, and often it is advisiable to use more than one method to completely remove continuum emission from the data. As usual, simpler methods are faster but less accurate. Important factors affecting their performance include:
- the fractional bandwidth over which one needs to subtract the continuum
- the quality of the calibration
- the distance of the continuum sources from the phase centre
- the brightness of the spectral line relative to the continuum

Below we describe these different methods, their advantages and disadvantages, and how the above factors come into play.

## 2. Cube-based continuum subtraction

If no continuum has been subtracted from the visibilities $V(u,v,\nu)$, each sightline ($l,m$) of the dirty cube $I^\mathrm{D}(l,m,\nu)$ will in principle include some level of continuum emission $I^\mathrm{D}_\mathrm{c}(l,m,\nu)\ne0$. This includes both emission at the position of real continuum sources and emission corresponding to their PSF sidelobes. The sidelobes contribution would disappear if one could clean the cube before subtracting the continuum. We will discuss this case below but, for the moment, assume that the cube is dirty.

### 2.1. Working with dirty cubes

The basic idea of this method is to estimate and remove the continuum component $I^\mathrm{D}_\mathrm{c}(l,m,\nu)$ of the dirty cube along each sightline ($l,m$) independently. This can be done by modelling it with a low order polynomial; that is, fit and subtract $I^\mathrm{D}_\mathrm{c,model}(l,m,\nu)=\sum_{n=0}^{N} a_n(l,m)\ \nu^n$ to the line-free channels. Since for a dirty cube the PSF spreads any line emission/absorption at a given channel to all spatial pixels of that channel, the line-free channel selection must be applied to all $(l,m)$.

In general, for smaller fractional bandwidths the variation of $I^\mathrm{D}_\mathrm{c}(l,m,\nu)$ with frequency is more limited and, therefore, the order $N$ of the polynomial can be smaller. The limiting case is one where it is sufficient to take the average of all line-free channels (or a $0^\mathrm{th}$-order polynomial fit) as an estimate of a frequency-independent continuum.

For a correct choice of $N$, note that the variation of $I^\mathrm{D}_\mathrm{c}(l,m,\nu)$ with frequency is determined not only by the intrinsic continuum spectrum of the sky $S_\mathrm{c}(l,m,\nu)$ but also by the variation of the primary beam $A(l,m,\nu)$ with frequency. The latter is a decreasing function of frequency (within the main lobe) and, therefore, has the effect of decreasing the spectral slope of the observed sources. Because of the primary beam two identical sources at different positions within the beam will in general have a different observed spectrum.

An additional effect is that of the PSF. At a position of the cube where most of the continuum flux comes from the sidelobes of a nearby source, the observed flux density variation with frequency depends critically on the structure of the PSF $B(l,m,\nu)$ and its variation with frequency (which may be in part due to frequency-dependent flagging). The resulting frequency dependence of $I^\mathrm{D}_\mathrm{c}(l,m,\nu)$ can be a lot more complex than that at the position of a real continuum source. This means that even when a low-$N$ approximation is valid for a source, it may be inaccurate for its sidelobes. In other words, this method of continuum subtraction results in an error which depends on the distance from the continuum source being subtracted as well as on the 3D PSF pattern $B(l,m,\nu)$. Cornwell, Uson & Addad (1992) give a formal discussion of this error for the case in which $I^\mathrm{D}_\mathrm{c}(l,m,\nu)$ is a linear function of frequency. Clearly, the lower the PSF sidelobe level, the lower the error.

Note that, in cubes made with MIRIAD, the pixel size scales with 1/frequency and, therefore, the PSF pattern does not change with frequency in the $(x,y,z)$ cube pixel grid. Therefore, for a source close to the phase centre the aforementioned error is much less of an issue. Far from the phase centre however this scaling means that sources move radially in $(x,y)$ as the frequency changes, and this complicates cube-based continuum subtraction further. In other words, this "trick" reduces the error as a function of distance from the continuum source but introduces a new error which depends on the distance from the phase centre.

### 2.2. Line-free channel selection and deconvolved cubes

The selection of line- and RFI-free channels is critical for a correct continuum subtraction. However, this is not always straightforward when working with a dirty cube. For example, the continuum emission (sources and sidelobes) may be much brighter than the line emission/absorption, and cleaning may be required before the line-free channels are identified.

A natural question is whether, since deconvolution may be required to identify the line-free channels, the cube-based continuum subtraction method could be applied directly to the deconvolved cube $I(l,m,\nu)$. This would have the advantage that the PSF sidelobes have been removed and, therefore, the aforementioned continuum subtraction error would be minimised. Furthermore, since the line emission/absorption has been cleaned too, and is now localised to a few small regions within each channel, the line-free channel selection could be made position dependent. This could be easily achieved by including a simple outlier rejection algorithm in the polynomial fit, and would maximise the number of fitted channels along each sightline. (In practice, this is not possible in current implementation of the cube-based continuum subtraction -- e.g., CASA and MIRIAD.)

One issue with deconvolving individual channels of $I^\mathrm{D}(l,m,\nu)$ before continuum subtraction is that deconvolution is non-linear and, therefore, leaves residuals and artefacts which vary from channel to channel. This would be particularly true for bright continuum emission and in the presence of significant calibration errors. The following step of continuum subtraction would not remove these artefacts. The final result may then be worse than one in which continuum subtraction is performed before deconvolution. In other words, continuum subtraction on the dirty cube $I^\mathrm{D}(l,m,\nu)$ is much more robust against calibration errors.

## 3. Visibility-based continuum subtraction

UVLIN approach. Advantage that it is independent of g(t) calibration. Limitation of distance from phase centre.

## 4. Model-based continuum subtraction

Modelling with MFS, then subtraction. Limited by accuracy of calibration and of continuum model.

## 5. Combining the above approaches

E.g., subtract a model, especially for distant sources, then UVLIN.

### Bibliography

<a href="http://adsabs.harvard.edu/abs/1992A%26A...258..583C">Cornwell, Uson & Addad 1992, A&A, 258, 583</a>

<a href="http://adsabs.harvard.edu/abs/1999ASPC..180..229R">Rupen 1999, ASPC, 180, 229</a>

<a href="http://adsabs.harvard.edu/abs/1994A%26AS..107...55S">Salt 1994, A&AS, 107, 55</a>

<a href="http://adsabs.harvard.edu/abs/1983ApJ...267..528V">van Gorkom & Ekers 1983, ApJ, 267, 528</a>

<a href="http://adsabs.harvard.edu/abs/1986syim.conf..177V">van Gorkom & Ekers 1986, Synthesis imaging, 177</a>

<a href="http://adsabs.harvard.edu/abs/1989ASPC....6..341V">van Gorkom & Ekers 1989, ASPC, 6, 341</a>

<a href="http://adsabs.harvard.edu/abs/1990A%26A...239L...5V">van Langevelde & Cotton 1990, A&A, 239L, 5

<a href="http://adsabs.harvard.edu/abs/2015MNRAS.453.2399W">Wang et al. 2015, MNRAS, 453, 2399</a>

### Chapter Editors
* Name 1
* Name 2

#### Chapter Contributors

* Name 3 (**1.0**, **1.1**)
* Name 4 (**1.1**)

***

   * Next: [1.1 Somename 2](01_01_somename2.ipynb)
